<a href="https://colab.research.google.com/github/SunSlick2/MoveEmails/blob/main/Online_Archive_Root_to_Inbox_Migrator_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import win32com.client
import pythoncom
import logging
import time
from datetime import datetime
import json
import os
import tkinter as tk
from tkinter import messagebox

class EmailMigrator:
    # Error codes consistent with server-side throttling/MAPI limits
    # -2147352567 is the generic COM error
    # -2147220731 is often the specific MAPI limit code
    SERVER_THROTTLE_ERRORS = [-2147352567, -2147220731]

    def __init__(self):
        self.setup_logging()
        self.migration_report = {
            'start_time': None,
            'end_time': None,
            'source_store': 'N/A',
            'source_folder': 'N/A',
            'destination_folder': 'N/A',
            'total_items_found': 0,
            'total_successful': 0,
            'total_failed': 0
        }

    def setup_logging(self):
        """Setup comprehensive logging for the migration."""
        log_dir = "migration_logs"
        os.makedirs(log_dir, exist_ok=True)
        log_filename = os.path.join(log_dir, f'online_archive_migration_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

        logging.basicConfig(
            filename=log_filename,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(levelname)s: %(message)s')
        console.setFormatter(formatter)
        logging.getLogger().addHandler(console)

        file_handler = logging.FileHandler(log_filename)
        file_handler.setLevel(logging.DEBUG)
        file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logging.getLogger().addHandler(file_handler)

        logging.info(f"Logging to: {log_filename}")

    def find_store_by_display_name(self, namespace, store_name):
        """
        Finds an Outlook store (mailbox or PST) by its display name.
        """
        for store in namespace.Stores:
            if store.DisplayName == store_name:
                logging.info(f"Store found: '{store.DisplayName}'")
                return store
        return None

    def get_remaining_items(self, source_folder):
        """Fetches the list of items in the source folder at the time of calling."""
        try:
            return list(source_folder.Items)
        except Exception as e:
            logging.error(f"Error accessing items list in source folder: {e}")
            return []

    def migrate_items(self, source_folder, destination_folder, items_to_move, start_index):
        """
        Moves a list of items from a source folder to a destination folder,
        starting from the given index, with aggressive throttling.
        Returns the number of successful moves and any error encountered.
        """
        moved_count_in_session = 0

        # --- Throttling Configuration ---
        THROTTLE_LIMIT = 50  # Move 50 items
        THROTTLE_DELAY = 5   # Wait for 5 seconds after 50 moves

        for i in range(start_index, len(items_to_move)):
            item = items_to_move[i]

            try:
                item.Move(destination_folder)
                moved_count_in_session += 1

                # Apply per-batch throttling
                if moved_count_in_session % THROTTLE_LIMIT == 0:
                    logging.info(f"Pausing for {THROTTLE_DELAY} seconds after {moved_count_in_session} moves...")
                    time.sleep(THROTTLE_DELAY)

            except pythoncom.com_error as com_err:
                error_code = com_err.args[0]
                # Check for known server-side throttling errors
                if error_code in self.SERVER_THROTTLE_ERRORS or (com_err.args[2] and com_err.args[2][5] in self.SERVER_THROTTLE_ERRORS):
                    logging.error(f"Server Throttle Detected (Error Code: {error_code}). Requiring session reset.")
                    # Return immediately with current successful count and the error
                    return moved_count_in_session, com_err
                else:
                    logging.warning(f"Failed to move item '{getattr(item, 'Subject', 'N/A')}' (Unknown COM Error): {com_err}")
                    self.migration_report['total_failed'] += 1
            except Exception as e:
                logging.warning(f"Failed to move item '{getattr(item, 'Subject', 'N/A')}': {e}")
                self.migration_report['total_failed'] += 1

        # Return successful if the entire list was processed without a fatal error
        return moved_count_in_session, None


    def run_migration(self):
        """Main function to run the migration process with connection reset logic."""
        self.migration_report['start_time'] = datetime.now().isoformat()

        # --- Hardcoded Paths ---
        ONLINE_ARCHIVE_NAME = "Online Archive - ghi.jkl@def.com"

        # Variables to track progress across multiple sessions
        total_items_found = 0
        total_items_moved_global = 0

        # Main loop for migration and session reset
        while True:
            outlook = None
            try:
                logging.info("-" * 40)
                logging.info(f"Starting new Outlook session. Total moved so far: {total_items_moved_global}")
                pythoncom.CoInitialize()
                outlook = win32com.client.Dispatch("Outlook.Application")
                namespace = outlook.GetNamespace("MAPI")

                # Step 1: Find the online archive store
                archive_store = self.find_store_by_display_name(namespace, ONLINE_ARCHIVE_NAME)
                if not archive_store:
                    raise ValueError(f"Online Archive '{ONLINE_ARCHIVE_NAME}' not found. Please ensure it is open in Outlook.")

                self.migration_report['source_store'] = archive_store.DisplayName

                # Step 2: Get the source and destination folders
                source_folder = archive_store.GetRootFolder()

                try:
                    destination_folder = source_folder.Folders['Inbox']
                except Exception:
                    raise ValueError("Inbox folder not found in the Online Archive root.")

                self.migration_report['source_folder'] = source_folder.FolderPath
                self.migration_report['destination_folder'] = destination_folder.FolderPath

                # Step 3: Initial setup and confirmation
                if total_items_moved_global == 0:
                    items_list = self.get_remaining_items(source_folder)
                    total_items_found = len(items_list)
                    self.migration_report['total_items_found'] = total_items_found

                    print("\n" + "=" * 50)
                    print("CONFIRMATION: You are about to MOVE ALL emails from:")
                    print(f"Source: {self.migration_report['source_folder']}")
                    print(f"To Destination: {self.migration_report['destination_folder']}")
                    print(f"Total items detected: {total_items_found}")
                    print("\nThis action will MOVE items and CANNOT BE UNDONE.")
                    print("The script is now throttled and RESILIENT to server errors.")
                    print("=" * 50)

                    confirm = input("Type 'CONFIRM' to proceed with the migration: ")
                    if confirm.strip().upper() != 'CONFIRM':
                        logging.warning("Migration cancelled by user.")
                        print("Operation cancelled.")
                        return False
                else:
                    # In a retry loop, items_list must be re-fetched to get fresh COM objects
                    items_list = self.get_remaining_items(source_folder)

                # Check if migration is complete
                if len(items_list) == 0 and total_items_moved_global == total_items_found:
                    logging.info("Migration complete: Source folder is empty and all items have been accounted for.")
                    break # Exit the while loop

                # The start_index for the new session is always 0 because items_list only contains remaining items
                items_to_move = items_list
                start_index = 0

                # Step 4: Start the migration for this session
                successful_moves_this_session, error = self.migrate_items(
                    source_folder, destination_folder, items_to_move, start_index
                )

                total_items_moved_global += successful_moves_this_session
                self.migration_report['total_successful'] = total_items_moved_global

                # Step 5: Check for error and handle restart
                if error:
                    logging.error(f"Migration session failed after moving {successful_moves_this_session} items due to server throttle.")
                    logging.info("Initiating 60 second cool-down and connection reset...")
                    time.sleep(60)
                    # The loop will restart automatically due to the break/except block logic

                elif total_items_moved_global == total_items_found:
                    logging.info("Migration complete: All found items have been moved successfully.")
                    break # Exit the while loop if all initial items are moved

                elif len(items_list) == 0:
                     logging.info("Migration complete: Source folder is now empty.")
                     break # Exit the while loop if the folder is empty

                # If no error, but not complete, something is wrong with the counts or loop logic, so we break/log to avoid infinite loop
                if not error and total_items_moved_global < total_items_found:
                    logging.warning(f"Session processed {successful_moves_this_session} items, but {total_items_found - total_items_moved_global} remain. Re-fetching items and restarting session.")
                    continue


            except Exception as e:
                # Catch any unexpected critical error (like store not found) and exit gracefully
                logging.error(f"Critical error during migration: {e}", exc_info=True)
                messagebox.showerror("Migration Error", f"A critical error occurred. Check logs for details: {e}")
                break # Exit the while loop on critical failure

            finally:
                # Crucially, clean up the COM objects in every iteration
                if outlook:
                    del outlook
                pythoncom.CoUninitialize()

        # Final reporting outside the loop
        self.migration_report['end_time'] = datetime.now().isoformat()
        self.generate_report()

        final_success = self.migration_report['total_failed'] == 0
        if final_success:
            logging.info("FINAL STATUS: Migration completed successfully!")
            return True
        else:
            logging.warning(f"FINAL STATUS: Migration completed with {self.migration_report['total_failed']} errors. Check the log file.")
            return False

    def generate_report(self):
        """Generates a comprehensive report of the migration."""
        report_dir = "migration_reports"
        os.makedirs(report_dir, exist_ok=True)
        report_filename = os.path.join(report_dir, f'migration_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json')

        with open(report_filename, 'w') as f:
            json.dump(self.migration_report, f, indent=2, default=str)

        logging.info(f"Migration report saved to: {report_filename}")

        print("\n=== MIGRATION SUMMARY ===")
        print(f"Source Folder: {self.migration_report['source_folder']}")
        print(f"Destination Folder: {self.migration_report['destination_folder']}")
        print(f"Total Items Found (Initial): {self.migration_report['total_items_found']}")
        print(f"Total Items Moved (Successful): {self.migration_report['total_successful']}")
        print(f"Total Items Failed (Skipped): {self.migration_report['total_failed']}")
        print("=========================")

# Usage
if __name__ == "__main__":
    print("\n" + "=" * 60)
    print("  Online Archive Root to Inbox Migrator (Resilient)  ")
    print("=" * 60)
    print("This tool will move all emails from your Online Archive's")
    print("root folder to its Inbox folder.")
    print("\n" + "=" * 60)

    migrator = EmailMigrator()
    migrator.run_migration()